In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [ ]:
!pip install --upgrade tensorflow==1.15

     |████████████████████████████████| 412.3MB 33kB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 512kB 47.1MB/s 
     |████████████████████████████████| 3.8MB 30.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=ee53ccd80ba51e56bc407eb7e990c32822dc9ecfa3a4aa137dba3b2bf0c0cf3e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0


In [ ]:
layer = {
    'input': 1,
    'output': 1
}

# VARIABLE PEMBANTU
batch_size = 2
time_step = 20
epoch = 5000

In [ ]:
def load_dataset():
    dataset = pd.read_csv('Avocado_sorted.csv', index_col='Date')
    dataset.index = pd.to_datetime(dataset.index)
    return dataset

In [ ]:
def get_batch(dataset, time_step, batch_size):
    input_batch = np.zeros(shape=(batch_size, time_step, layer['input']))
    output_batch = np.zeros(shape=(batch_size, time_step, layer['output']))

    for i in range(len(dataset)):
        point = np.random.randint(0, len(dataset) - time_step)
        input_batch += dataset[point : point + time_step]
        output_batch += dataset[point + 1 : point + time_step + 1]

    return input_batch, output_batch

In [ ]:
dataset = load_dataset()

FileNotFoundError: ignored

In [ ]:
dataset

In [ ]:
# TRAIN TEST SPLIT
split = int(len(dataset) * 0.8)
train_dataset = dataset[:split]
test_dataset = dataset[split:]

In [ ]:
# NORMALISASI
scaler = MinMaxScaler().fit(train_dataset)
normalize_train_dataset = scaler.transform(train_dataset)

In [ ]:
normalize_train_dataset

In [ ]:
# BUILDING MODEL
cell = tf.nn.rnn_cell.BasicRNNCell(10, activation = tf.nn.relu)
cell = tf.contrib.rnn.OutputProjectionWrapper(cell, layer['output'], activation = tf.nn.relu)

In [ ]:
cell

In [ ]:
input_feature = tf.placeholder(tf.float32, [None, time_step, layer['input']])
input_target = tf.placeholder(tf.float32, [None, time_step, layer['output']])

In [ ]:
output, _ = tf.nn.dynamic_rnn(cell, input_feature, dtype = tf.float32)

In [ ]:
# MSE
loss = tf.reduce_mean(0.5 * (input_target - output) ** 2)
train = tf.train.AdamOptimizer(0.0001).minimize(loss)

In [ ]:
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(1, epoch + 1):
        input_batch, output_batch = get_batch(normalize_train_dataset, time_step, batch_size)
        print(input_batch.shape)
        feed = { 
            input_feature: input_batch,
            input_target: output_batch
        }
        sess.run(train, feed_dict = feed)

        if i % 200 == 0:
            print("Iteration: {}, Loss: {}" .format(i, sess.run(loss, feed_dict = feed)))

    saver.save(sess, './rnn-model.ckpt') # checkpoint

In [ ]:
with tf.Session() as sess:
    seed_data = list(normalize_train_dataset)
    saver.restore(sess, './rnn-model.ckpt')

    for i in range(len(test_dataset)):
        input_batch = np.array(seed_data[-time_step:]).reshape(1, time_step, layer['input'])
        print(input_batch.shape)
        feed = {
            input_feature: input_batch
        }
        predict = sess.run(output, feed_dict = feed)
        seed_data.append(predict[0, -1, 0])

    predict_result = scaler.inverse_transform(np.array(seed_data[-len(test_dataset):]).reshape(len(test_dataset), 1))
    test_dataset['Prediction'] = predict_result
    test_dataset.plot()
    plt.show()